In [1]:
import robosuite 
import dexmimicgen  

import h5py 
import imageio 
import numpy as np
import time 
import os
import json

'''

1. Loading dual robot environment
'''

# def get_env_metadata_from_dataset(dataset_path, ds_format="robomimic"):
#     """
#     Retrieves env metadata from dataset.

#     Args:
#         dataset_path (str): path to dataset

#     Returns:
#         env_meta (dict): environment metadata. Contains 3 keys:

#             :`'env_name'`: name of environment
#             :`'type'`: type of environment, should be a value in EB.EnvType
#             :`'env_kwargs'`: dictionary of keyword arguments to pass to environment constructor
#     """
#     dataset_path = os.path.expanduser(dataset_path)
#     f = h5py.File(dataset_path, "r")
#     if ds_format == "robomimic":
#         env_meta = json.loads(f["data"].attrs["env_args"])
#     else:
#         raise ValueError
#     f.close()
#     return env_meta

from vishal_dev.utils import get_env_metadata_from_dataset

dataset_path = "/home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/dexmimicgen/datasets/two_arm_box_cleanup.hdf5"

env_meta = get_env_metadata_from_dataset(dataset_path)
print(env_meta)

write_to_video = False

env_kwargs = env_meta['env_kwargs']
env_kwargs["env_name"] = env_meta["env_name"]
env_kwargs["has_renderer"] = True
env_kwargs["renderer"] = "mjviewer"
env_kwargs["has_offscreen_renderer"] = write_to_video # False # write_video
env_kwargs["use_camera_obs"] = False

env_kwargs.pop("env_lang")

env = robosuite.make(**env_kwargs)

[robosuite WARNING] No private macro file found! (macros.py:57)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:58)
[robosuite WARNING] To setup, run: python /home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/robosuite/robosuite/scripts/setup_macros.py (macros.py:59)
[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properly, otherwise you will not be able to use the default IK controller setting for GR1 robot. (__init__.py:40)


{'env_name': 'TwoArmBoxCleanup', 'env_version': '1.5.1', 'type': 1, 'env_kwargs': {'robots': ['PandaDexRH', 'PandaDexLH'], 'controller_configs': {'type': 'BASIC', 'body_parts': {'right': {'type': 'OSC_POSE', 'input_max': 1, 'input_min': -1, 'output_max': [0.05, 0.05, 0.05, 0.5, 0.5, 0.5], 'output_min': [-0.05, -0.05, -0.05, -0.5, -0.5, -0.5], 'kp': 150, 'damping_ratio': 1, 'impedance_mode': 'fixed', 'kp_limits': [0, 300], 'damping_ratio_limits': [0, 10], 'position_limits': None, 'orientation_limits': None, 'uncouple_pos_ori': True, 'input_type': 'delta', 'input_ref_frame': 'base', 'interpolation': None, 'ramp_ratio': 0.2, 'gripper': {'type': 'GRIP', 'use_action_scaling': False}}}}, 'translucent_robot': False, 'env_configuration': 'parallel', 'reward_shaping': False, 'camera_names': ['agentview', 'robot0_eye_in_hand', 'robot1_eye_in_hand'], 'camera_heights': 84, 'camera_widths': 84, 'has_renderer': False, 'has_offscreen_renderer': True, 'ignore_done': True, 'use_object_obs': True, 'use_

In [2]:
import pose_utils_vishal_from_mimicgen as PoseUtils
import robosuite.utils.transform_utils as T

In [3]:
'''
Extracting a demonstration from the dataset
'''

data = h5py.File(dataset_path, "r")
print(f"Keys in the dataset: {data['data'].keys()}")
print(f"Number of demonstrations in the dataset: {len(data['data'].keys())}")
demo_id = 'demo_15'
print(f"Length of demo {demo_id}: {data['data'][demo_id]['obs']['robot0_eef_pos'].shape[0]}")
print(f"Observations shape: {data['data'][demo_id]['obs'].keys()}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Actions: {data['data'][demo_id]['actions']}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Action at step 0: {data['data'][demo_id]['actions'][0]}")

action_trajectory = data['data'][demo_id]['actions']
obs_trajectory = data['data'][demo_id]['obs']
states_trajectory = data['data'][demo_id]['states']

eef_poses = data['data'][demo_id]['datagen_info']['eef_pose']
gripper_actions = data['data'][demo_id]['datagen_info']['gripper_action']
target_eef_poses = data['data'][demo_id]['datagen_info']['target_pose']

Keys in the dataset: <KeysViewHDF5 ['demo_0', 'demo_1', 'demo_10', 'demo_100', 'demo_1000', 'demo_1001', 'demo_1002', 'demo_1003', 'demo_1004', 'demo_1005', 'demo_1006', 'demo_1007', 'demo_1008', 'demo_1009', 'demo_101', 'demo_1010', 'demo_1011', 'demo_1012', 'demo_1013', 'demo_1014', 'demo_1015', 'demo_102', 'demo_103', 'demo_104', 'demo_105', 'demo_106', 'demo_107', 'demo_108', 'demo_109', 'demo_11', 'demo_110', 'demo_111', 'demo_112', 'demo_113', 'demo_114', 'demo_115', 'demo_116', 'demo_117', 'demo_118', 'demo_119', 'demo_12', 'demo_120', 'demo_121', 'demo_122', 'demo_123', 'demo_124', 'demo_125', 'demo_126', 'demo_127', 'demo_128', 'demo_129', 'demo_13', 'demo_130', 'demo_131', 'demo_132', 'demo_133', 'demo_134', 'demo_135', 'demo_136', 'demo_137', 'demo_138', 'demo_139', 'demo_14', 'demo_140', 'demo_141', 'demo_142', 'demo_143', 'demo_144', 'demo_145', 'demo_146', 'demo_147', 'demo_148', 'demo_149', 'demo_15', 'demo_150', 'demo_151', 'demo_152', 'demo_153', 'demo_154', 'demo_155'

In [4]:
from vishal_dev.utils import just_reset, reset_to_state, reset_to

In [5]:
ep = demo_id
states = data["data/{}/states".format(ep)][()]
initial_state = dict(states=states[0])
initial_state["model"] = data["data/{}".format(ep)].attrs["model_file"]
# if args.use_current_model:
initial_state["model"] = env.sim.model.get_xml()
initial_state["ep_meta"] = data["data/{}".format(ep)].attrs.get("ep_meta", None)

In [6]:
'''
Replay a demonstration
'''
# reset_to_state(env, states_trajectory[0])
# just_reset(env, states_trajectory[25])

for i in range(action_trajectory.shape[0]):
    action = action_trajectory[i]
    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)

In [7]:
max_dpos = 0.05
max_drot = 0.5

from vishal_dev.utils import eef_to_action_single_arm, eef_to_action

In [8]:
'''
Data class for source and transformed segments

inputs:
    arm1_src_traj: (N, 4, 4) numpy array
    arm2_src_traj: (N, 4, 4) numpy array
    starting_lid_pose: (4, 4) numpy array
    starting_box_pose: (4, 4) numpy array

to_maintain:
    arm1_src_segment1: (N, 4, 4) numpy array
    arm1_src_segment2: (N, 4, 4) numpy array
    arm2_src_segment1: (N, 4, 4) numpy array
    arm2_src_segment2: (N, 4, 4) numpy array
    starting_lid_pose: (4, 4) numpy array
    starting_box_pose: (4, 4) numpy array
    arm1_tgt_segment1: (N, 4, 4) numpy array
    arm1_tgt_segment2: (N, 4, 4) numpy array
    arm2_tgt_segment1: (N, 4, 4) numpy array
    arm2_tgt_segment2: (N, 4, 4) numpy array
    
    INTERPOLATED TRAJECTORIES
    arm1_interp_toseg1: (N, 4, 4) numpy array
    arm1_interp_toseg2: (N, 4, 4) numpy array
    arm2_interp_toseg1: (N, 4, 4) numpy array
    arm2_interp_toseg2: (N, 4, 4) numpy array
'''

'\nData class for source and transformed segments\n\ninputs:\n    arm1_src_traj: (N, 4, 4) numpy array\n    arm2_src_traj: (N, 4, 4) numpy array\n    starting_lid_pose: (4, 4) numpy array\n    starting_box_pose: (4, 4) numpy array\n\nto_maintain:\n    arm1_src_segment1: (N, 4, 4) numpy array\n    arm1_src_segment2: (N, 4, 4) numpy array\n    arm2_src_segment1: (N, 4, 4) numpy array\n    arm2_src_segment2: (N, 4, 4) numpy array\n    starting_lid_pose: (4, 4) numpy array\n    starting_box_pose: (4, 4) numpy array\n    arm1_tgt_segment1: (N, 4, 4) numpy array\n    arm1_tgt_segment2: (N, 4, 4) numpy array\n    arm2_tgt_segment1: (N, 4, 4) numpy array\n    arm2_tgt_segment2: (N, 4, 4) numpy array\n    \n    INTERPOLATED TRAJECTORIES\n    arm1_interp_toseg1: (N, 4, 4) numpy array\n    arm1_interp_toseg2: (N, 4, 4) numpy array\n    arm2_interp_toseg1: (N, 4, 4) numpy array\n    arm2_interp_toseg2: (N, 4, 4) numpy array\n'

In [9]:
# obfull[0].keys()

In [228]:
'''
Now prepare the data
'''
obfull = just_reset(env, states_trajectory[0])
src_lid_pos = data['data'][demo_id]['datagen_info']['object_poses']['lid'][0][:3, 3]
src_lid_mat = data['data'][demo_id]['datagen_info']['object_poses']['lid'][0][:3, :3].reshape(3, 3)
src_box_pos = data['data'][demo_id]['datagen_info']['object_poses']['box'][0][:3, 3]
src_box_mat = data['data'][demo_id]['datagen_info']['object_poses']['box'][0][:3, :3].reshape(3, 3)
src_lid_pose = PoseUtils.make_pose(src_lid_pos, src_lid_mat)
src_box_pose = PoseUtils.make_pose(src_box_pos, src_box_mat)
print("Starting lid pose: ", src_lid_pose)
print("Starting box pose: ", src_box_pose)

new_lid_pos = env.sim.data.body('lid_obj_root').xpos
new_lid_mat = env.sim.data.body('lid_obj_root').xmat.reshape(3, 3)
new_box_pos = env.sim.data.body('box_obj_root').xpos
new_box_mat = env.sim.data.body('box_obj_root').xmat.reshape(3, 3)
new_lid_pose = PoseUtils.make_pose(new_lid_pos, new_lid_mat)
new_box_pose = PoseUtils.make_pose(new_box_pos, new_box_mat)
print("New lid pose: ", new_lid_pose)
print("New box pose: ", new_box_pose)

cur_arm1_eef_pos = obfull[0]['robot0_eef_pos']
cur_arm1_eef_mat = obfull[0]['robot0_eef_quat']
cur_arm1_eef_mat = T.quat2mat(cur_arm1_eef_mat)
cur_arm1_eef_pose = PoseUtils.make_pose(cur_arm1_eef_pos, cur_arm1_eef_mat)
print("Current arm1 eef pose: ", cur_arm1_eef_pose)
cur_arm2_eef_pos = obfull[0]['robot1_eef_pos']
cur_arm2_eef_mat = obfull[0]['robot1_eef_quat']
cur_arm2_eef_mat = T.quat2mat(cur_arm2_eef_mat)
cur_arm2_eef_pose = PoseUtils.make_pose(cur_arm2_eef_pos, cur_arm2_eef_mat)
print("Current arm2 eef pose: ", cur_arm2_eef_pose)
cur_eef_poses = np.zeros((2, 4, 4))
cur_eef_poses[0, :, :] = cur_arm1_eef_pose
cur_eef_poses[1, :, :] = cur_arm2_eef_pose

Starting lid pose:  [[ 1.          0.          0.         -0.00800182]
 [ 0.          1.          0.          0.2       ]
 [ 0.          0.          1.          0.725     ]
 [ 0.          0.          0.          1.        ]]
Starting box pose:  [[ 1.          0.          0.         -0.00938105]
 [ 0.          1.          0.         -0.15      ]
 [ 0.          0.          1.          0.776     ]
 [ 0.          0.          0.          1.        ]]
New lid pose:  [[ 1.          0.          0.         -0.00903334]
 [ 0.          1.          0.          0.2       ]
 [ 0.          0.          1.          0.72476748]
 [ 0.          0.          0.          1.        ]]
New box pose:  [[ 1.00000000e+00  3.81907915e-25 -4.66219212e-14 -3.66817709e-02]
 [-4.55517054e-25  1.00000000e+00 -1.57885255e-12 -1.50000000e-01]
 [ 4.66219212e-14  1.57885255e-12  1.00000000e+00  7.74126732e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Current arm1 eef pose:  [[ 0.99710412 -0.01112

In [229]:
'''0.55977141
Transforming the source trajectories to the new object positions
'''

'''
Transforming two segments one by one

Process:
1. Transform segment 1 using object pose at start of segment 1
2. Interpolate between starting pose to starting pose of segment 1
3. Transform segment 2 using object pose at start of segment 2
4. Interpolate between end of segment 1 to starting pose of segment 2

'''

from pose_utils_vishal_from_mimicgen import transform_source_data_segment_using_object_pose
from pose_utils_vishal_from_mimicgen import interpolate_poses
from pose_utils_vishal_from_mimicgen import make_pose
from pose_utils_vishal_from_mimicgen import unmake_pose

def transform_two_segments_for_one_arm(
        segment1,
        segment2,
        src_obj1_pose,
        cur_obj1_pose,
        src_obj2_pose,
        cur_obj2_pose,
        starting_eef_pose,
    ):
    '''Transform 2 segments of trajectory from source to current based on object poses

    Args:
        segment1 (list): list of eef_poses for segment 1 (T, 4, 4)
        segment2 (list): list of eef_poses for segment 2 (T, 4, 4)
        src_obj1_pose (np.ndarray): source object pose (4, 4)
        cur_obj1_pose (np.ndarray): current object pose (4, 4)
        src_obj2_pose (np.ndarray): source object pose (4, 4)
        cur_obj2_pose (np.ndarray): current object pose (4, 4)
        starting_eef_pose (list): starting eef pose (4, 4)
    Returns:
        
    '''
    # Transform segments into their corresponding target object poses
    transformed_segment1 = transform_source_data_segment_using_object_pose(
        src_eef_poses=segment1,
        src_obj_pose=src_obj1_pose,
        obj_pose=cur_obj1_pose,
    )
    transformed_segment2 = transform_source_data_segment_using_object_pose(
        src_eef_poses=segment2,
        src_obj_pose=src_obj2_pose,
        obj_pose=cur_obj2_pose,
    )

    # Interpolate between starting pose to starting pose of segment 1
    interp_poses0, num_steps = interpolated_poses = interpolate_poses(
        pose_1=starting_eef_pose,
        pose_2=transformed_segment1[0],
        num_steps=10,
        # step_size=0.1, # one of step size or num_steps is needed
    )

    # Interpolate between end of segment 1 to starting pose of segment 2
    interp_poses1, num_steps = interpolated_poses = interpolate_poses(
        pose_1=transformed_segment1[-1],
        pose_2=transformed_segment2[0],
        num_steps=10,
        # step_size=0.1, # one of step size or num_steps is needed
    )

    # Create a dict of all trajectories 
    overall_trajectory = {
        'interp_to_segment1': interp_poses0,
        'segment1': transformed_segment1,
        'interp_to_segment2': interp_poses1,
        'segment2': transformed_segment2,
    }

    return overall_trajectory

def get_transformed_segments_for_both_arms(
        segment1,
        segment2,
        src_obj1_pose,
        cur_obj1_pose,
        src_obj2_pose,
        cur_obj2_pose,
        starting_eef_pose,
    ):
    '''

    args:
        segment1 (np.ndarray): (N, 2, 4, 4)
        segment2 (np.ndarray): (M, 2, 4, 4) 
        src_obj1_pose (np.ndarray): (4, 4)
        cur_obj1_pose (np.ndarray): (4, 4)
        src_obj2_pose (np.ndarray): (4, 4)
        cur_obj2_pose (np.ndarray): (4, 4)
        starting_eef_pose (list): list of two eef poses [(4, 4), (4, 4)]
    returns:
        dict: {
            "arm1": {
                "overall_trajectory": overall_trajectory_arm1 (dict): {
                    'interp_to_segment1': interp_poses0,
                    'segment1': transformed_segment1,
                    'interp_to_segment2': interp_poses1,
                    'segment2': transformed_segment2,
                },
                "starting_eef_pose": starting_eef_pose_arm1 (list): [(4, 4)]
            },
            "arm2": {
                "overall_trajectory": overall_trajectory_arm2 (dict): {
                    'interp_to_segment1': interp_poses0,
                    'segment1': transformed_segment1,
                    'interp_to_segment2': interp_poses1,
                    'segment2': transformed_segment2,
                },
                "starting_eef_pose": starting_eef_pose_arm2 (list): [(4, 4)]
            }
        }
    '''

    arm1_segment1 = segment1[:, 0, :, :]
    arm1_segment2 = segment2[:, 0, :, :]
    arm2_segment1 = segment1[:, 1, :, :]
    arm2_segment2 = segment2[:, 1, :, :]
    starting_eef_pose_arm1 = starting_eef_pose[0] # [starting_eef_pose[0][0], starting_eef_pose[1][0]]
    starting_eef_pose_arm2 = starting_eef_pose[1] # [starting_eef_pose[0][1], starting_eef_pose[1][1]]

    overall_trajectory_arm1 = transform_two_segments_for_one_arm(
        segment1=arm1_segment1,
        segment2=arm1_segment2,
        src_obj1_pose=src_obj1_pose,
        cur_obj1_pose=cur_obj1_pose,
        src_obj2_pose=src_obj2_pose,
        cur_obj2_pose=cur_obj2_pose,
        starting_eef_pose=starting_eef_pose_arm1,
    )

    overall_trajectory_arm2 = transform_two_segments_for_one_arm(
        segment1=arm2_segment1,
        segment2=arm2_segment2,
        src_obj1_pose=src_obj1_pose,
        cur_obj1_pose=cur_obj1_pose,
        src_obj2_pose=src_obj2_pose,
        cur_obj2_pose=cur_obj2_pose,
        starting_eef_pose=starting_eef_pose_arm2,
    )

    return {
        "arm1": {
            "overall_trajectory": overall_trajectory_arm1,
            "starting_eef_pose": starting_eef_pose_arm1
        },
        "arm2": {
            "overall_trajectory": overall_trajectory_arm2,
            "starting_eef_pose": starting_eef_pose_arm2
        }
    }


In [230]:
cur_eef_poses

array([[[ 0.99710412, -0.0111229 ,  0.0752307 , -0.1149491 ],
        [-0.01151279, -0.99992256,  0.00475083, -0.24922874],
        [ 0.07517201, -0.00560319, -0.99715496,  1.09918484],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.99373069,  0.0045496 ,  0.11170773, -0.11766847],
        [ 0.00527893, -0.99996641, -0.00623403,  0.2428523 ],
        [ 0.11167565,  0.00678464, -0.99372132,  1.10387679],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]])

In [231]:
from vishal_dev.utils import segment_trajectory

segments = segment_trajectory(data, demo_id)

src_arm_segment1 = target_eef_poses[:segments[0][-1]+1, :]
src_arm_segment2 = target_eef_poses[segments[0][-1]+1:segments[1][-1]+1, :]

full_trajectory_outs = get_transformed_segments_for_both_arms(
    segment1=src_arm_segment1,
    segment2=src_arm_segment2,
    src_obj1_pose=src_lid_pose,
    cur_obj1_pose=new_lid_pose,
    src_obj2_pose=src_box_pose,
    cur_obj2_pose=new_box_pose,
    starting_eef_pose=cur_eef_poses,
)

<HDF5 dataset "lid_off_ground": shape (238,), type "<i8">


In [232]:
# transformed_data = get_transformed_segments_for_both_arms(
#     segment1=eef_poses[0],
#     segment2=eef_poses[1],
#     src_obj1_pose=src_lid_pose,
#     cur_obj1_pose=new_lid_pose,
#     src_obj2_pose=src_box_pose,
#     cur_obj2_pose=new_box_pose,
#     starting_eef_pose=target_eef_poses,
# )

In [233]:
'''
Execution order/logic for trajectories:

# 1. Interpolate to initial poses for each arm in parallel
N, M = len(arm1_interp_toseg1), len(arm2_interp_toseg1)

for i in range(max(N, M)):
    initialize action with zeros
    if i < N:
        set action[0:12] to arm1_interp_toseg1[i]
    if i < M:
        set action[12:24] to arm2_interp_toseg1[i]
    env.step(action)

# 2. Execute segment1 for each arm in parallel. This segment will be of same length for both arms

for i in range(len(arm1_segment1)):
    initialize action with zeros
    set action[0:12] to arm1_segment1[i]
    set action[12:24] to arm2_segment1[i]
    env.step(action)

# 3. Interpolate to initial poses of segment 2 for each arm in parallel
N, M = len(arm1_interp_toseg2), len(arm2_interp_toseg2)

for i in range(max(N, M)):
    initialize action with zeros
    if i < N:
        set action[0:12] to arm1_interp_toseg2[i]
    if i < M:
        set action[12:24] to arm2_interp_toseg2[i]
    env.step(action)

# 4. Execute segment2 for each arm in parallel. This segment will be of same length for both arms

for i in range(len(arm1_segment2)):
    initialize action with zeros
    set action[0:12] to arm1_segment2[i]
    set action[12:24] to arm2_segment2[i]
    env.step(action)

'''

'\nExecution order/logic for trajectories:\n\n# 1. Interpolate to initial poses for each arm in parallel\nN, M = len(arm1_interp_toseg1), len(arm2_interp_toseg1)\n\nfor i in range(max(N, M)):\n    initialize action with zeros\n    if i < N:\n        set action[0:12] to arm1_interp_toseg1[i]\n    if i < M:\n        set action[12:24] to arm2_interp_toseg1[i]\n    env.step(action)\n\n# 2. Execute segment1 for each arm in parallel. This segment will be of same length for both arms\n\nfor i in range(len(arm1_segment1)):\n    initialize action with zeros\n    set action[0:12] to arm1_segment1[i]\n    set action[12:24] to arm2_segment1[i]\n    env.step(action)\n\n# 3. Interpolate to initial poses of segment 2 for each arm in parallel\nN, M = len(arm1_interp_toseg2), len(arm2_interp_toseg2)\n\nfor i in range(max(N, M)):\n    initialize action with zeros\n    if i < N:\n        set action[0:12] to arm1_interp_toseg2[i]\n    if i < M:\n        set action[12:24] to arm2_interp_toseg2[i]\n    env.

In [234]:
'''
Reset environment to new starting poses
'''



'\nReset environment to new starting poses\n'

In [235]:
'''
Generate data

'''
interp_arm1_toseg1 = full_trajectory_outs["arm1"]["overall_trajectory"]["interp_to_segment1"]
interp_arm1_toseg2 = full_trajectory_outs["arm1"]["overall_trajectory"]["interp_to_segment2"]
interp_arm2_toseg1 = full_trajectory_outs["arm2"]["overall_trajectory"]["interp_to_segment1"]
interp_arm2_toseg2 = full_trajectory_outs["arm2"]["overall_trajectory"]["interp_to_segment2"]
seg1_arm1 = full_trajectory_outs["arm1"]["overall_trajectory"]["segment1"]
seg1_arm2 = full_trajectory_outs["arm2"]["overall_trajectory"]["segment1"]
seg2_arm1 = full_trajectory_outs["arm1"]["overall_trajectory"]["segment2"]
seg2_arm2 = full_trajectory_outs["arm2"]["overall_trajectory"]["segment2"]

In [236]:
print(interp_arm1_toseg1[-2], interp_arm1_toseg1[-1])
print(seg1_arm1[0], seg1_arm1[1])

[[-0.93742295 -0.28483045 -0.20027368 -0.11994934]
 [-0.20200102 -0.02361973  0.9791005  -0.24498973]
 [-0.28360798  0.9582869  -0.03539451  1.09376049]
 [ 0.          0.          0.          1.        ]] [[-0.99597645 -0.08945754 -0.00531572 -0.12044937]
 [-0.00507369 -0.00293271  0.99998283 -0.24456583]
 [-0.08947159  0.99598632  0.00246703  1.09321805]
 [ 0.          0.          0.          1.        ]]
[[-0.99597645 -0.08945754 -0.00531572 -0.12044937]
 [-0.00507369 -0.00293271  0.99998283 -0.24456583]
 [-0.08947159  0.99598632  0.00246703  1.09321805]
 [ 0.          0.          0.          1.        ]] [[-9.94672835e-01 -1.03078630e-01 -8.64436828e-04 -1.18323881e-01]
 [-6.25523821e-04 -2.35007456e-03  9.99997043e-01 -2.44109549e-01]
 [-1.03080357e-01  9.94670434e-01  2.27307717e-03  1.09538084e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [237]:
# PoseUtils.unmake_pose(interp_arm1_toseg1[i])[0] #, 0]

In [238]:
def eef_to_action_single_arm(eef_pos, eef_rot_mats, original_action, prev_eef_pos, prev_eef_quat, max_dpos=0.05, max_drot=0.5, cur_rot_from_env=False, arm_index=0):
    """
    Convert end-effector pose target actions to simulation-executable action space.

    args:
        eef_pos (np.ndarray): target end-effector position (3,)
        eef_rot_mats (np.ndarray): target end-effector orientation as rotation matrices (3, 3)
        original_action (np.ndarray): original action to copy other dimensions from (15,)
        prev_eef_pos (np.ndarray): previous end-effector position (3,)
        prev_eef_quat (np.ndarray): previous end-effector orientation as quaternion (4,)
        max_dpos (float): maximum position delta
        max_drot (float): maximum rotation delta in axis-angle representation
        cur_rot_from_env (bool): rotation matrices from environment are slightly different from that ones from the dataset.
            If true, these rotation matrices must be adjusted before computing the delta rotation. Otherwise, the delta rotation
            will be incorrect.
        arm_index (int): index of the arm (0 or 1) (arm 0 has a different form of rotation matrix transformation as compared to arm 1 when current rotation is from env)
    """
    max_dpos = 0.05
    max_drot = 0.5

    action = np.copy(original_action)

    print(eef_pos.shape, eef_rot_mats.shape, prev_eef_pos.shape, prev_eef_quat.shape, action.shape)
    

    # Set robot action in position space
    action[0:3] = eef_pos - prev_eef_pos #[0]
    action[0:3] = np.clip(action[0:3] / max_dpos, -1., 1.)

    # Set robot action in rotation space
    curr_rot = T.quat2mat(prev_eef_quat)
    target_rot = eef_rot_mats#[0] # T.quat2mat(prev_eef_quat[0])
    if cur_rot_from_env and arm_index == 0:
        curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
        curr_rot = -1 * curr_rot
    if cur_rot_from_env and arm_index == 1:
        curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
        curr_rot[:, 1] = -1 * curr_rot[:, 1]
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    action[3:6] = delta_rotation  
    return action

In [239]:
# print(N, M)

In [240]:
action_trajectory[80:100, 6:12] # .shape

array([[0.        , 0.        , 0.        , 0.        , 0.26772322,
        1.57079633],
       [0.        , 0.        , 0.        , 0.        , 0.21230339,
        1.57079633],
       [0.        , 0.        , 0.        , 0.        , 0.40048983,
        1.57079633],
       [0.        , 0.        , 0.27938188, 0.        , 0.31084185,
        1.57079633],
       [0.        , 0.55977141, 0.71498286, 0.        , 0.23069122,
        1.57079633],
       [0.12035175, 0.75792219, 0.81231528, 0.24081942, 0.1725213 ,
        1.57079633],
       [0.        , 0.66333467, 0.72755346, 0.08957932, 0.287578  ,
        1.57079633],
       [0.24521817, 0.77802507, 0.78930712, 0.        , 0.17791856,
        1.57079633],
       [0.58666448, 0.9052433 , 0.89333283, 0.26155026, 0.24961576,
        1.57079633],
       [0.8839965 , 1.07489475, 1.03642752, 0.59659285, 0.2647425 ,
        1.57079633],
       [0.98473961, 1.12896475, 1.07757703, 0.71044474, 0.40918166,
        1.57079633],
       [1.10382147, 1

In [241]:
prev_action = action_trajectory[0] 
action = prev_action.copy()

eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm1_toseg1[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm1_toseg1[i])[1],
                original_action=action[0:12], # action_trajectory[0],
                prev_eef_pos=obs['robot0_eef_pos'],
                prev_eef_quat=obs['robot0_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=False, #cTrue
            )

IndexError: index 131 is out of bounds for axis 0 with size 12

In [242]:
'''
Execute interpolation to initial poses for each arm in parallel
'''

# obs = obfull[0]
# reset_to(env, initial_state) # Reset this to some other state
# reset_to_state(env, states_trajectory[0])
# obfull = just_reset(env, initial_state) #  --- IGNORE ---

obs = obfull[0].copy()

N, M = len(interp_arm1_toseg1), len(interp_arm2_toseg1)

evolved_poses_interp_arm1 = []
evolved_poses_interp_arm2 = []

prev_action = action_trajectory[0] #/ N

for i in range(max(N, M)):

    action = prev_action.copy()

    if i == 0:
        if i < N:
            action[0:12] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm1_toseg1[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm1_toseg1[i])[1],
                original_action=action[0:12], # action_trajectory[0],
                prev_eef_pos=obs['robot0_eef_pos'],
                prev_eef_quat=obs['robot0_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=False, #cTrue
            )
        if i < M:
            action[12:24] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm2_toseg1[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm2_toseg1[i])[1],
                original_action=action[12:24], # action_trajectory[0],
                prev_eef_pos=obs['robot1_eef_pos'],
                prev_eef_quat=obs['robot1_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=False, #cTrue
            )

    else:   
        if i < N:
            action[0:12] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm1_toseg1[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm1_toseg1[i])[1],
                original_action=action[0:12], # action_trajectory[0],
                prev_eef_pos=obs['robot0_eef_pos'],
                prev_eef_quat=obs['robot0_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=True,
                arm_index=0
            )   
        if i < M:
            action[12:24] = eef_to_action_single_arm(
                eef_pos=PoseUtils.unmake_pose(interp_arm2_toseg1[i])[0],
                eef_rot_mats=PoseUtils.unmake_pose(interp_arm2_toseg1[i])[1],
                original_action=action[12:24], # action_trajectory[0],
                prev_eef_pos=obs['robot1_eef_pos'],
                prev_eef_quat=obs['robot1_eef_quat'],
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=True,
                arm_index=1
            )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    print(f"Step {i}, EEF1 pos: {obs['robot0_eef_pos']}, EEF2 pos: {obs['robot1_eef_pos']}")

    evolved_poses_interp_arm1.append(PoseUtils.make_pose(obs['robot0_eef_pos'], T.quat2mat(obs['robot0_eef_quat'])))
    evolved_poses_interp_arm2.append(PoseUtils.make_pose(obs['robot1_eef_pos'], T.quat2mat(obs['robot1_eef_quat'])))

    env.render()
    time.sleep(0.05)


(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
Step 0, EEF1 pos: [-0.11491721 -0.24936746  1.09920711], EEF2 pos: [-0.1176452   0.24298892  1.10389246]
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
Step 1, EEF1 pos: [-0.10836794 -0.25080214  1.10571454], EEF2 pos: [-0.11770254  0.25704891  1.10437042]
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
Step 2, EEF1 pos: [-0.09864802 -0.24939093  1.11618523], EEF2 pos: [-0.11439406  0.28887521  1.10923344]
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
Step 3, EEF1 pos: [-0.0924614  -0.24304759  1.12372541], EEF2 pos: [-0.10953456  0.30909112  1.116629  ]
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
Step 4, EEF1 pos: [-0.09054898 -0.23381384  1.12725799], EEF2 pos: [-0.10606838  0.31689706  1.12211964]
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
Step 5, EEF1 pos: [-0.09209635 -0.22361723  1.12749691], EEF2 pos: [-0.10611459  0.3173929   1.12282451]
(3,) (3, 3) (3,) (4,) (12,)
(3,) (

In [243]:
print(f"Current arm1 eef pose: ", cur_arm1_eef_pose)
print(f"Current arm2 eef pose: ", cur_arm2_eef_pose)


Current arm1 eef pose:  [[ 0.99710412 -0.0111229   0.0752307  -0.1149491 ]
 [-0.01151279 -0.99992256  0.00475083 -0.24922874]
 [ 0.07517201 -0.00560319 -0.99715496  1.09918484]
 [ 0.          0.          0.          1.        ]]
Current arm2 eef pose:  [[ 0.99373069  0.0045496   0.11170773 -0.11766847]
 [ 0.00527893 -0.99996641 -0.00623403  0.2428523 ]
 [ 0.11167565  0.00678464 -0.99372132  1.10387679]
 [ 0.          0.          0.          1.        ]]


In [244]:
print(f"Arm 1 seg 1 starting pose: ",PoseUtils.unmake_pose(seg1_arm1[i]))

Arm 1 seg 1 starting pose:  (array([-0.10698422, -0.23793659,  1.09546537]), array([[-0.93343365, -0.34155568,  0.10973306],
       [ 0.10350088,  0.0364775 ,  0.99396024],
       [-0.34349555,  0.9391534 ,  0.00130198]]))


In [245]:
print(f"Arm 2 seg 1 starting pose: ", PoseUtils.unmake_pose(seg1_arm2[i]))

Arm 2 seg 1 starting pose:  (array([-0.10836379,  0.26237877,  1.1056188 ]), array([[ 0.95676299, -0.28817168,  0.03951793],
       [ 0.03254044, -0.02896511, -0.99905062],
       [ 0.28904273,  0.95714059, -0.01833552]]))


In [246]:
print(f"Ending pose of interpolation to seg 1 for arm 1: ", PoseUtils.unmake_pose(interp_arm1_toseg1[-1]))
print(f"Ending pose of interpolation to seg 1 for arm 2: ", PoseUtils.unmake_pose(interp_arm2_toseg1[-1]))

Ending pose of interpolation to seg 1 for arm 1:  (array([-0.12044937, -0.24456583,  1.09321805]), array([[-0.99597645, -0.08945754, -0.00531572],
       [-0.00507369, -0.00293271,  0.99998283],
       [-0.08947159,  0.99598632,  0.00246703]]))
Ending pose of interpolation to seg 1 for arm 2:  (array([-0.12276258,  0.22776699,  1.10249474]), array([[ 0.98700808, -0.13165436, -0.09209875],
       [-0.09198753,  0.00693941, -0.99573598],
       [ 0.13173209,  0.99127139, -0.00526131]]))


In [247]:
evolved_poses_interp_arm1

[array([[ 0.99709206, -0.01158566,  0.07532065, -0.11491721],
        [-0.01197392, -0.99991712,  0.0047052 , -0.24936746],
        [ 0.07525989, -0.0055934 , -0.99714814,  1.09920711],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.99473305,  0.03720031,  0.09551072, -0.10836794],
        [ 0.03833683, -0.99921409, -0.01009144, -0.25080214],
        [ 0.09506024,  0.01369986, -0.99537743,  1.10571454],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.98014524,  0.14125668,  0.13914691, -0.09864802],
        [ 0.14687414, -0.98867183, -0.03091305, -0.24939093],
        [ 0.13320397,  0.05073637, -0.98978894,  1.11618523],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.94917952,  0.24950849,  0.19184308, -0.0924614 ],
        [ 0.26099357, -0.96464239, -0.03671413, -0.24304759],
        [ 0.17589943,  0.08491811, -0.98073891,  1.12372541],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),


In [248]:
interp_arm1_toseg1

array([[[ 9.97104119e-01, -1.11229029e-02,  7.52306953e-02,
         -1.14949098e-01],
        [-1.15127880e-02, -9.99922558e-01,  4.75082826e-03,
         -2.49228737e-01],
        [ 7.51720145e-02, -5.60318585e-03, -9.97154957e-01,
          1.09918484e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]],

       [[ 9.69938502e-01, -2.09864742e-01, -1.23191326e-01,
         -1.15449122e-01],
        [-2.08163089e-01, -9.77730523e-01,  2.66716665e-02,
         -2.48804836e-01],
        [-1.26045311e-01, -2.25984679e-04, -9.92024571e-01,
          1.09864240e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]],

       [[ 8.65272903e-01, -3.95240841e-01, -3.08362645e-01,
         -1.15949147e-01],
        [-3.88228993e-01, -9.17485288e-01,  8.65978780e-02,
         -2.48380935e-01],
        [-3.17145138e-01,  4.47845301e-02, -9.47319115e-01,
          1.09809997e+00],
        [ 0.00000000e+00,  0.00000000e+

In [249]:
interp_arm2_toseg1

array([[[ 0.99373069,  0.0045496 ,  0.11170773, -0.11766847],
        [ 0.00527893, -0.99996641, -0.00623403,  0.2428523 ],
        [ 0.11167565,  0.00678464, -0.99372132,  1.10387679],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.99426888, -0.01930771,  0.10515042, -0.11813157],
        [-0.00351558, -0.98892921, -0.14834487,  0.24148091],
        [ 0.10685056,  0.14712502, -0.98332941,  1.10375115],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.99456576, -0.04199372,  0.0952653 , -0.11859467],
        [-0.01291065, -0.95772716, -0.28738743,  0.24010952],
        [ 0.10330666,  0.28459575, -0.95306481,  1.10362551],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.99461527, -0.06304526,  0.08225418, -0.11905777],
        [-0.02271446, -0.90699736, -0.42052274,  0.23873813],
        [ 0.10111632,  0.41639   , -0.90354547,  1.10349987],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],



In [250]:
for i in range(10):
    print(f"Seg 1 arm 1 pose i: {i}, {PoseUtils.unmake_pose(seg1_arm1[i])}")
    print(f"Seg 1 arm 2 pose i: {i}, {PoseUtils.unmake_pose(seg1_arm2[i])}")

for i in range(10):
    print(f"Seg 1 arm 1 pose i: {N - 10 + i}, {PoseUtils.unmake_pose(seg1_arm1[N - 10 + i])}")
    print(f"Seg 1 arm 2 pose i: {N - 10 + i}, {PoseUtils.unmake_pose(seg1_arm2[N - 10 + i])}")

Seg 1 arm 1 pose i: 0, (array([-0.12044937, -0.24456583,  1.09321805]), array([[-0.99597645, -0.08945754, -0.00531572],
       [-0.00507369, -0.00293271,  0.99998283],
       [-0.08947159,  0.99598632,  0.00246703]]))
Seg 1 arm 2 pose i: 0, (array([-0.12276258,  0.22776699,  1.10249474]), array([[ 0.98700808, -0.13165436, -0.09209875],
       [-0.09198753,  0.00693941, -0.99573598],
       [ 0.13173209,  0.99127139, -0.00526131]]))
Seg 1 arm 1 pose i: 1, (array([-0.11832388, -0.24410955,  1.09538084]), array([[-9.94672835e-01, -1.03078630e-01, -8.64436828e-04],
       [-6.25523821e-04, -2.35007456e-03,  9.99997043e-01],
       [-1.03080357e-01,  9.94670434e-01,  2.27307717e-03]]))
Seg 1 arm 2 pose i: 1, (array([-0.12016209,  0.23438431,  1.10488991]), array([[ 0.98789968, -0.13690216, -0.07288364],
       [-0.07307011,  0.00366961, -0.99732006],
       [ 0.13680273,  0.99057778, -0.00637825]]))
Seg 1 arm 1 pose i: 2, (array([-0.11619839, -0.24365327,  1.09754363]), array([[-0.99316327,

In [251]:
print(f"Arm 1 current eef pose: ", obs['robot0_eef_pos'], T.quat2mat(obs['robot0_eef_quat']))
print(f"Arm 2 current eef pose: ", obs['robot1_eef_pos'], T.quat2mat(obs['robot1_eef_quat']))

Arm 1 current eef pose:  [-0.14001429 -0.24016453  1.07443837] [[ 0.74750751  0.37868154  0.5457406 ]
 [ 0.41437155 -0.90796249  0.06245237]
 [ 0.51916158  0.17945577 -0.83562364]]
Arm 2 current eef pose:  [-0.16701085  0.23422377  1.07020162] [[ 0.97356971  0.06561398  0.21876206]
 [-0.04344896 -0.88714214  0.45944676]
 [ 0.22421913 -0.45680842 -0.86084386]]


In [252]:
'''
Execute transformed synced trajectory for each arm in parallel - segment 1

obs carried forward from previous loop
'''

# Assert both segments are of same length
assert len(seg1_arm1) == len(seg1_arm2)

N = len(seg1_arm1)

prev_action = action_trajectory[0]

for i in range(N):
    action = prev_action.copy()

    # if i == 0:
    #     action[0:12] = eef_to_action_single_arm(
    #         eef_pos=PoseUtils.unmake_pose(seg1_arm1[i])[0],
    #         eef_rot_mats=PoseUtils.unmake_pose(seg1_arm1[i])[1],
    #         original_action=action_trajectory[i, 0:12], # action_trajectory[0],
    #         prev_eef_pos=obs['robot0_eef_pos'],
    #         prev_eef_quat=obs['robot0_eef_quat'], 
    #         max_dpos=max_dpos,
    #         max_drot=max_drot,
    #         cur_rot_from_env=False, #cTrue
    #     )
    
    #     action[12:24] = eef_to_action_single_arm(
    #         eef_pos=PoseUtils.unmake_pose(seg1_arm2[i])[0],
    #         eef_rot_mats=PoseUtils.unmake_pose(seg1_arm2[i])[1],
    #         original_action=action_trajectory[i, 12:24], # action_trajectory[0],
    #         prev_eef_pos=obs['robot1_eef_pos'],
    #         prev_eef_quat=obs['robot1_eef_quat'],
    #         max_dpos=max_dpos,
    #         max_drot=max_drot,
    #         cur_rot_from_env=False, #cTrue
    #     )

    # else:   
    action[0:12] = eef_to_action_single_arm(
        eef_pos=PoseUtils.unmake_pose(seg1_arm1[i])[0],
        eef_rot_mats=PoseUtils.unmake_pose(seg1_arm1[i])[1],
        original_action=action_trajectory[i, 0:12], # action_trajectory[0],
        prev_eef_pos=obs['robot0_eef_pos'],
        prev_eef_quat=obs['robot0_eef_quat'],
        max_dpos=max_dpos,
        max_drot=max_drot,
        cur_rot_from_env=True,
        arm_index=0
    )   

    action[12:24] = eef_to_action_single_arm(
        eef_pos=PoseUtils.unmake_pose(seg1_arm2[i])[0],
        eef_rot_mats=PoseUtils.unmake_pose(seg1_arm2[i])[1],
        original_action=action_trajectory[i, 12:24], # action_trajectory[0],
        prev_eef_pos=obs['robot1_eef_pos'],
        prev_eef_quat=obs['robot1_eef_quat'],
        max_dpos=max_dpos,
        max_drot=max_drot,
        cur_rot_from_env=True,
        arm_index=1
    )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)


(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,

In [253]:
'''
Execute interpolation to initial poses of segment 2 for each arm in parallel
'''

# obs = obfull[0]
# reset_to(env, initial_state) # Reset this to some other state
# reset_to_state(env, states_trajectory[0])
# obfull = just_reset(env, initial_state) #  --- IGNORE ---

obs = obfull[0].copy()

N, M = len(interp_arm1_toseg2), len(interp_arm2_toseg2)

# prev_action = action_trajectory[0]

for i in range(max(N, M)):

    action = prev_action.copy()

    # if i == 0:
    #     if i < N:
    #         action[0:12] = eef_to_action_single_arm(
    #             eef_pos=PoseUtils.unmake_pose(interp_arm1_toseg2[i])[0],
    #             eef_rot_mats=PoseUtils.unmake_pose(interp_arm1_toseg2[i])[1],
    #             original_action=prev_action[0:12], # action_trajectory[0],
    #             prev_eef_pos=obs['robot0_eef_pos'],
    #             prev_eef_quat=obs['robot0_eef_quat'],
    #             max_dpos=max_dpos,
    #             max_drot=max_drot,
    #             cur_rot_from_env=False, #cTrue
    #         )
    #     if i < M:
    #         action[12:24] = eef_to_action_single_arm(
    #             eef_pos=PoseUtils.unmake_pose(interp_arm2_toseg2[i])[0],
    #             eef_rot_mats=PoseUtils.unmake_pose(interp_arm2_toseg2[i])[1],
    #             original_action=prev_action[12:24], # action_trajectory[0],
    #             prev_eef_pos=obs['robot1_eef_pos'],
    #             prev_eef_quat=obs['robot1_eef_quat'],
    #             max_dpos=max_dpos,
    #             max_drot=max_drot,
    #             cur_rot_from_env=False, #cTrue
    #         )

    # else:   
    if i < N:
        action[0:12] = eef_to_action_single_arm(
            eef_pos=PoseUtils.unmake_pose(interp_arm1_toseg2[i])[0],
            eef_rot_mats=PoseUtils.unmake_pose(interp_arm1_toseg2[i])[1],
            original_action=prev_action[0:12], # action_trajectory[0],
            prev_eef_pos=obs['robot0_eef_pos'],
            prev_eef_quat=obs['robot0_eef_quat'],
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True,
            arm_index=0
        )   
    if i < M:
        action[12:24] = eef_to_action_single_arm(
            eef_pos=PoseUtils.unmake_pose(interp_arm2_toseg2[i])[0],
            eef_rot_mats=PoseUtils.unmake_pose(interp_arm2_toseg2[i])[1],
            original_action=prev_action[12:24], # action_trajectory[0],
            prev_eef_pos=obs['robot1_eef_pos'],
            prev_eef_quat=obs['robot1_eef_quat'],
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True,
            arm_index=1
        )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)


(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)


In [ ]:
# seg2_arm1.shape

(238, 4, 4)

In [254]:
'''
Execute transformed synced trajectory for each arm in parallel - segment 2

obs carried forward from previous loop
'''

# Assert both segments are of same length
assert len(seg2_arm1) == len(seg2_arm2)

N = len(seg2_arm1)

prev_action = action_trajectory[0]

for i in range(N):
    action = prev_action.copy()

    # if i == 0:
    #     action[0:12] = eef_to_action_single_arm(
    #         eef_pos=PoseUtils.unmake_pose(seg2_arm1[i])[0],
    #         eef_rot_mats=PoseUtils.unmake_pose(seg2_arm1[i])[1],
    #         original_action=action_trajectory[i+len(segments[0])][0:12], # action_trajectory[0],
    #         prev_eef_pos=obs['robot0_eef_pos'],
    #         prev_eef_quat=obs['robot0_eef_quat'],
    #         max_dpos=max_dpos,
    #         max_drot=max_drot,
    #         cur_rot_from_env=False, #cTrue
    #     )
    
    #     action[12:24] = eef_to_action_single_arm(
    #         eef_pos=PoseUtils.unmake_pose(seg2_arm2[i])[0],
    #         eef_rot_mats=PoseUtils.unmake_pose(seg2_arm2[i])[1],
    #         original_action=action_trajectory[i+len(segments[0])][12:24], # action_trajectory[0],
    #         prev_eef_pos=obs['robot1_eef_pos'],
    #         prev_eef_quat=obs['robot1_eef_quat'],
    #         max_dpos=max_dpos,
    #         max_drot=max_drot,
    #         cur_rot_from_env=False, #cTrue
    #     )

    # else:   
    action[0:12] = eef_to_action_single_arm(
        eef_pos=PoseUtils.unmake_pose(seg2_arm1[i])[0],
        eef_rot_mats=PoseUtils.unmake_pose(seg2_arm1[i])[1],
        original_action=action_trajectory[i+len(segments[0])-1][0:12], # action_trajectory[0],
        prev_eef_pos=obs['robot0_eef_pos'],
        prev_eef_quat=obs['robot0_eef_quat'],
        max_dpos=max_dpos,
        max_drot=max_drot,
        cur_rot_from_env=True,
        arm_index=0
    )   

    action[12:24] = eef_to_action_single_arm(
        eef_pos=PoseUtils.unmake_pose(seg2_arm2[i])[0],
        eef_rot_mats=PoseUtils.unmake_pose(seg2_arm2[i])[1],
        original_action=action_trajectory[i+len(segments[0])-1][12:24], # action_trajectory[0],
        prev_eef_pos=obs['robot1_eef_pos'],
        prev_eef_quat=obs['robot1_eef_quat'],
        max_dpos=max_dpos,
        max_drot=max_drot,
        cur_rot_from_env=True,
        arm_index=1
    )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)


(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,) (12,)
(3,) (3, 3) (3,) (4,